In [1]:
#@Imports
import sys
import os
path = os.path.abspath('../..')
if path not in sys.path:
  sys.path.insert(0, path)

%load_ext autoreload

import collections
import concurrent.futures
import datetime
import json
import termcolor
import re

import numpy as np

from IPython import display

from concordia.agents import basic_agent
from concordia.components import agent as components
from concordia import components as generic_components
from concordia.associative_memory import associative_memory
from concordia.associative_memory import blank_memories
from concordia.associative_memory import formative_memories
from concordia.associative_memory import importance_function
from concordia.clocks import game_clock
from concordia.components import game_master as gm_components
from concordia.environment import game_master
from concordia.metrics import goal_achievement
from concordia.metrics import common_sense_morality
from concordia.metrics import opinion_of_others
from concordia.utils import html as html_lib
from concordia.utils import measurements as measurements_lib
from concordia.utils import plotting

In [2]:
#@Model Setup
local_models = True

if local_models:
    # Setup LLM
    from concordia.language_model import ollama_model
    model = ollama_model.OllamaLanguageModel(
        model_name='llama3:70b',
        streaming=True
    )

    # Setup measurements and clock
    measurements = measurements_lib.Measurements()
    time_step = datetime.timedelta(minutes=20)
    SETUP_TIME = datetime.datetime(hour=20, year=2024, month=10, day=1)

    START_TIME = datetime.datetime(hour=18, year=2024, month=10, day=2)
    clock = game_clock.MultiIntervalClock(
        start=SETUP_TIME,
        step_sizes=[time_step, datetime.timedelta(seconds=10)])

    # Setup sentence encoder
    from sentence_transformers import SentenceTransformer
    st5_model = SentenceTransformer('sentence-transformers/sentence-t5-base')
    embedder = st5_model.encode

else:
    from concordia.language_model import gpt_model
    from concordia.language_model import gcloud_model
    # @title Language Model - pick your model and provide keys
    CLOUD_PROJECT_ID = '' #@param {type: 'string'}
    GPT_API_KEY = '' #@param {type: 'string'}
    GPT_MODEL_NAME = '' #@param {type: 'string'}

    USE_CLOUD = True #@param {type: 'boolean'}

    if USE_CLOUD:
        model = gcloud_model.CloudLanguageModel(project_id= CLOUD_PROJECT_ID)
    else:
        model = gpt_model.GptLanguageModel(api_key=GPT_API_KEY, model_name=GPT_MODEL_NAME)

#@title Importance models
importance_model = importance_function.ConstantImportanceModel()
importance_model_gm = importance_function.ConstantImportanceModel()


In [3]:
#@Shared Memories

scenario_premise = [

    (
        'Alice returns late from work and finds her friend Dorothy at her doorstep. '
        + 'Dorothy has to move out of her house tonight and needs someone to help her move.'
    ),
]
# The generic context will be used for the NPC context. It reflects general
# knowledge and is possessed by all characters.
shared_context = model.sample_text(
    'Summarize the following passage in a concise and insightful fashion:\n'
    + '\n'.join(scenario_premise)
    + '\n'
    + 'Summary:'
)

Alice comes home from work to find her friend Dorothy unexpectedly waiting at her doorstep, , who urgently needs Alice's help to move out of her house that very night.

In [4]:
#@title Creating character backgrounds, goals and traits. Modify to explore how it influences the outcomes
NUM_PLAYERS = 2

player_configs = [
    formative_memories.AgentConfig(
        name='Alice',
        gender='female',
        goal='Alice is exhausted after work, and she needs to help her grandmother all day the next day.',
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
    formative_memories.AgentConfig(
        name='Dorothy',
        gender='female',
        goal=(
            "Dorothy has planned on Bob helping her move, but Bob didn't show up. "
        ),
        context=shared_context,
        traits='responsibility: medium; aggression: medium',
    ),
]

blank_memory_factory = blank_memories.MemoryFactory(
    model=model,
    embedder=embedder,
    importance=importance_model.importance,
    clock_now=clock.now,
)

formative_memory_factory = formative_memories.FormativeMemoryFactory(
    model=model,
    shared_memories=scenario_premise,
    blank_memory_factory_call=blank_memory_factory.make_blank_memory,
)

mem = formative_memory_factory.make_memories(player_configs[0])

Alice grew up in a loving family, where she was instilled with a strong sense of responsibility and a moderate dose of aggression - just enough to make her determined, but not overly confrontational. As a child, she would help her mother with household chores and care for her younger siblings, always eager to lend a hand. This trait followed her into adulthood, where she became a dedicated nurse at a local hospital. Her typical day was filled with the hustle and bustle of the emergency room, where she thrived on the fast-paced environment and the sense of fulfillment that came with helping others.

Despite her demanding job, Alice always made time for her loved ones and friends. She cherished her relationships, especially with Dorothy, who had been by her side since college days. So, when Dorothy unexpectedly showed up at her doorstep one evening, frantic and in need of urgent help to move out of her house that very night, Alice didn't hesitate. She dropped everything to assist her dea

Number of generated formative episodes (1) does not match number of formative ages (5).


In [5]:
%autoreload 2
from examples.custom_components import tpb_llama3 as tpb

behav = tpb.Behaviour(
  "behav",
  model=model,
  memory=mem,
  player_config=player_configs[0],
  num_behavs=2
)

behav.update()

Here are two potential behaviors Alice can take in response to the situation:

1. **Help Dorothy immediately**: Given her past experiences and traits, Alice is likely to prioritize helping her friend in need over her own exhaustion and plans for the next day. She may put aside her fatigue and help Dorothy move out of her house that night, just as she did when Dorothy needed help in similar situations before.

2. **Ask Dorothy if it can wait until morning**: While Alice values her friendship with Dorothy and wants to help, she is also exhausted from work and has a commitment to help her grandmother the next day. She may ask Dorothy if it's possible to postpone the move until morning when she'll be more rested and better equipped to assist, rather than sacrificing her own well-being and potentially compromising her ability to help her grandmother.

In [25]:
%autoreload 2
from examples.custom_components import tpb_llama3 as tpb

attitude = tpb.Attitude(
    "conseq",
    model=model,
    memory=mem,
    player_config=player_configs[0],
    components=[behav],
)
attitude.update()

Here are the potential consequences of Alice asking Dorothy if it can wait until morning:

**Positive Consequences:**

1. **Alice gets a good night's rest**: By postponing the move to the next day, Alice can get a full night's sleep and wake up feeling rested and refreshed, which will help her be more productive and focused when helping her grandmother the next day. (Value: 8, Likelihood: 80)
2. **Alice is better equipped to help Dorothy**: With a good night's rest, Alice will be more energized and focused, allowing her to provide better assistance to Dorothy during the move. (Value: 7, Likelihood: 70)
3. **Alice sets a healthy boundary**: By prioritizing her own needs and setting a boundary with Dorothy, Alice demonstrates self-care and assertiveness, which can have long-term benefits for her mental and emotional well-being. (Value: 6, Likelihood: 60)

**Negative Consequences:**

1. **Dorothy's crisis worsens**: If Dorothy is in an urgent situation and needs help immediately, delaying

TypeError: list indices must be integers or slices, not str

In [57]:
%autoreload 2
from examples.custom_components import tpb_llama3 as tpb

people = tpb.People(
    "people",
    model=model,
    memory=mem,
    player_config=player_configs[0],
    components=[behav]
)

people.update()

Here are 5 people who might have opinions about whether Alice should help Dorothy immediately:

1. **Alice's Mother**: Having seen Alice's kindness and responsibility since childhood, Alice's mother would likely approve of her helping Dorothy in need. (Approval: 8)
2. **Dorothy**: As the recipient of Alice's help, Dorothy would be extremely grateful and relieved if Alice assists her tonight. (Approval: 10)
3. **Alice's Grandmother**: Since Alice needs to help her grandmother all day tomorrow, Alice's grandmother might disapprove of Alice exhausting herself by helping Dorothy tonight. (Approval: -5)
4. **Timmy (Alice's Brother)**: Having been protected by Alice when he was bullied, Timmy would likely admire and approve of his sister's kindness towards others in need. (Approval: 9)
5. **Mrs. Johnson (the neighbor)**: As someone who benefited from Alice's kindness in the past, Mrs. Johnson would probably appreciate Alice's willingness to help Dorothy and approve of her actions. (Approval:

In [104]:
%autoreload 2
from examples.custom_components import tpb_llama3 as tpb

motiv = tpb.Motivation(
    "motiv",
    model=model,
    memory=mem,
    player_config=player_configs[0],
    components=[people]
)
motiv.update()

Motivation: 60(Motivation: 80)(Motivation: 60)Motivation: 85Motivation: 20Motivation: 80

In [105]:
%autoreload 2
from examples.custom_components import tpb_llama3 as tpb

norm = tpb.SubjectiveNorm(
    "norm",
    model=model,
    memory=mem,
    player_config=player_configs[0],
    components=[motiv]
)
norm.update()

In [106]:
norm.json()

[{'behaviour': '**Help Dorothy immediately**: Given her past experiences and traits, Alice is likely to prioritize helping her friend in need over her own exhaustion and plans for the next day. She may put aside her fatigue and help Dorothy move out of her house that night, just as she did when Dorothy needed help in similar situations before.',
  'people': [{'person': "Alice's Mother", 'approval': 8, 'motivation': 0.8},
   {'person': 'Dorothy', 'approval': 10, 'motivation': 0.85},
   {'person': "Alice's Grandmother", 'approval': -5, 'motivation': 0.8},
   {'person': 'Timmy', 'approval': 9, 'motivation': 0.6},
   {'person': 'Mrs. Johnson', 'approval': 7, 'motivation': 0.6}],
  'norm': 20.5},
 {'behaviour': "**Ask Dorothy if it can wait until morning**: While Alice values her friendship with Dorothy and wants to help, she is also exhausted from work and has a commitment to help her grandmother the next day. She may ask Dorothy if it's possible to postpone the move until morning when she

In [26]:
from typing import Iterable
line = iter([r'(?<=\d:\s)(.*)', r'(?<=: )(.*)'])


from typing import Iterable
from retry import retry

@retry(AttributeError, tries = 5)
def extract_value(line: str, patterns: Iterable[str]) -> int:
  pattern = next(patterns)
  value = re.search(
    pattern, line
  ).group(1).strip()

True